# Dataset Lending ETL

In [1]:
# Import dependencies
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
sns.set() 

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("max_rows", None)


In [2]:
# import datasets
# accepted applicants data file 
raw_accepted = pd.read_csv('../Archive/accepted_2007_to_2018Q4.csv') 
# rejected applicants data file
raw_rejected = pd.read_csv('../Archive/rejected_2007_to_2018Q4.csv')

In [3]:
#create data copy 
file_rejected = raw_rejected.copy()


In [4]:
#create data copy 
file_accepted = raw_accepted.copy()


# Analysis 1 .Binary classification model to accept or reject loan application

In [5]:
''' 
Only keep those columns that are going to be used for analysis 
 
Application Date - date does not play a role in getting rejected or accepted unless you beleive in Numerology 
Loan Title - For out initial model we have left this out since it did not add a lot of value to the rejected decison. 
Zip Code - We saw people getting accepted and rejected for teh same zipcodes so we think that zipcode does not have a big impact on loan application
Policy Code- This is our target column so we have it but made sure it was all '0'

'''
file_rejected = file_rejected[['Amount Requested', 'Risk_Score',
       'Debt-To-Income Ratio', 'Employment Length']]

In [6]:
# Extact the same columms from the  accepted applicant df to match the rejected data set. We will combine this dataset at a later set to make a complete data frame that will be used to train teh classification model. 

file_accepted = file_accepted[['loan_amnt', 'fico_range_low', 'fico_range_high', 'dti', 'emp_length']]
file_accepted.head()

,loan_amnt,fico_range_low,fico_range_high,dti,emp_length
0,3600.00000,675.00000,679.00000,5.91000,10+ years
1,24700.00000,715.00000,719.00000,16.06000,10+ years
2,20000.00000,695.00000,699.00000,10.78000,10+ years
3,35000.00000,785.00000,789.00000,17.06000,10+ years
4,10400.00000,695.00000,699.00000,25.37000,3 years


In [7]:
file_rejected['Label_target'] = 0


In [8]:
# Calculate the average FICO score for the high and low range of the applicant
file_accepted['average'] = (file_accepted['fico_range_low'] + file_accepted['fico_range_high'])*0.5

# Drop the FICO high and low scores 
file_accepted = file_accepted.drop(columns=['fico_range_low','fico_range_high'], axis=1)

In [9]:
#Create a target label column for accepted df
file_accepted['label_target']=1   


In [10]:
# rename columns for accepted and rejected df
file_accepted.rename(columns= {'loan_amnt': 'Loan_Amount', 'dti': 'Debt_income_ratio', 'average': 'Credit_Score', 'label_target': 'Loan_Status', 'emp_length':'Emp_Length'}, inplace=True)

file_rejected.rename(columns= {'Amount Requested': 'Loan_Amount', 'Debt-To-Income Ratio': 'Debt_income_ratio', 'Risk_Score': 'Credit_Score', 'Label_target': 'Loan_Status', 'Employment Length':'Emp_Length'}, inplace=True)

In [11]:
# rearrange columns in rejected df to concat with the accepted df
file_rejected = file_rejected[['Loan_Amount', 'Debt_income_ratio', 'Emp_Length', 'Credit_Score', 'Loan_Status']]


In [12]:
# remove % from Debt_income_ratio
file_rejected['Debt_income_ratio'] = file_rejected['Debt_income_ratio'].str.replace('%','')
file_rejected['Debt_income_ratio'] = pd.to_numeric(file_rejected['Debt_income_ratio'])
file_rejected.head()


,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,1000.00000,10.00000,4 years,693.00000,0
1,1000.00000,10.00000,< 1 year,703.00000,0
2,11000.00000,10.00000,1 year,715.00000,0
3,6000.00000,38.64000,< 1 year,698.00000,0
4,1500.00000,9.43000,< 1 year,509.00000,0


In [13]:
# Check number of NaNs in rejected dataset

file_rejected.isnull().sum()

Loan_Amount                 0
Debt_income_ratio           0
Emp_Length             951355
Credit_Score         18497630
Loan_Status                 0
dtype: int64

In [14]:
# Check number of NaNs in accepted dataset

file_accepted.isnull().sum()

Loan_Amount              33
Debt_income_ratio      1744
Emp_Length           146940
Credit_Score             33
Loan_Status               0
dtype: int64

In [15]:
# Drop NaNs from both datasets

file_accepted = file_accepted.dropna()
file_rejected = file_rejected.dropna()
print(f'There are {file_accepted.shape[0]} succesfull applications and {file_rejected.shape[0]} unsuccessful applications')

There are 2113648 succesfull applications and 8992595 unsuccessful applications


In [16]:
# Save cleaned data sets 
# file_accepted.to_csv('../Archive/accepted_data_clean', index=False)
# file_rejected.to_csv('../Archive/rejected_data_clean', index=False)

In [17]:
# Concatinate the two dataframes to single 
consolidated = pd.concat([file_accepted, file_rejected], axis=0).reset_index(drop=True)
consolidated.head()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,3600.00000,5.91000,10+ years,677.00000,1
1,24700.00000,16.06000,10+ years,717.00000,1
2,20000.00000,10.78000,10+ years,697.00000,1
3,35000.00000,17.06000,10+ years,787.00000,1
4,10400.00000,25.37000,3 years,697.00000,1


In [18]:
# Check unique values in Emp_length col
consolidated['Emp_Length'].value_counts()

< 1 year     8358386
10+ years     960253
5 years       365812
2 years       282099
3 years       248095
1 year        224209
4 years       186245
6 years       137652
7 years       122446
8 years       119819
9 years       101227
Name: Emp_Length, dtype: int64

In [19]:
# Remove the strings years, <,>,+ from Emp_length

consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('10+ years','10')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('< 1 year','0')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('1 year','1')
consolidated['Emp_Length'] = consolidated['Emp_Length'].str.replace(' years','')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace(' ','')
consolidated['Emp_Length'] = pd.to_numeric(consolidated['Emp_Length'])
consolidated['Emp_Length'].value_counts()

0     8358386
10     960253
5      365812
2      282099
3      248095
1      224209
4      186245
6      137652
7      122446
8      119819
9      101227
Name: Emp_Length, dtype: int64

In [20]:
# Get dataset summary and identify outliers
consolidated.describe()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
count,11106243.00000,11106243.00000,11106243.00000,11106243.00000,11106243.00000
mean,13132.42598,140.83387,1.55425,641.25526,0.19031
std,10653.05310,15988.09176,3.17176,87.03325,0.39255
min,0.00000,-1.00000,0.00000,0.00000,0.00000
25%,5000.00000,9.89000,0.00000,604.00000,0.00000
50%,10000.00000,19.65000,0.00000,653.00000,0.00000
75%,20000.00000,32.80000,0.00000,687.00000,0.00000
max,1400000.00000,50000031.49000,10.00000,990.00000,1.00000


In [21]:
#counting outliers
consolidated[(consolidated['Debt_income_ratio']>300) | (consolidated['Debt_income_ratio']<0) | (consolidated['Credit_Score']>850) | (consolidated['Credit_Score']<0)].count()

Loan_Amount          161845
Debt_income_ratio    161845
Emp_Length           161845
Credit_Score         161845
Loan_Status          161845
dtype: int64

In [22]:
#Clean up data outliers
consolidated.drop(consolidated[consolidated['Debt_income_ratio']>200].index, inplace = True)
consolidated.drop(consolidated[consolidated['Debt_income_ratio']<0].index, inplace = True)
consolidated.drop(consolidated[consolidated['Credit_Score']<0].index, inplace = True)
consolidated.drop(consolidated[consolidated['Credit_Score']>850].index, inplace = True)
consolidated.shape

(10901852, 5)

In [46]:
# Inital Run by sampling the small subset of data  to make sure the cide runs 
consolidated = consolidated.sample(frac=0.01, replace=False, random_state=1)
consolidated.shape

(10902, 5)

### Data Visualization

In [47]:
#Since dataset is very large, we will extract a small sample from our datasaet and plot 
# sample_df = consolidated.sample(frac=0.1, replace=False, random_state=1)

# plt.scatter(sample_df['Debt_income_ratio'],sample_df['Credit_Score'],c=sample_df['Loan_Status'])
# plt.show()

In [48]:
# sample_df['Debt_income_ratio'].plot.density()

In [49]:
# sample_df['Credit_Score'].plot.density()

### Removing Class Imbalance
 

### Method 1: Oversampling
This involves duplicating the minority class. This can potentially lead to overfitting or biasing towards some of minority class outliers/data. 

In [50]:
# Import dependencies
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from sklearn.preprocessing import StandardScaler
from collections import Counter

In [51]:
# Preprocess the data by using StandardScaler

y = consolidated['Loan_Status']
X = consolidated.drop('Loan_Status',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

scaler = StandardScaler()

scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f' The count for y_train before oversample is:{Counter(y_train)}')

 The count for y_train before oversample is:Counter({0: 6543, 1: 1633})


In [52]:
# Oversampling the data
over = RandomOverSampler(sampling_strategy='minority') # 'minority' can be replaced by a float < 1
X_train_over,y_train_over = over.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after oversample is:{Counter(y_train_over)}')

 The count for y_train after oversample is:Counter({0: 6543, 1: 6543})


### Method 2: Undersampling
This involves removing samples from the majority class. This can potentially lead to losing some of teh important infromation from the dataset.

In [53]:
# Undersampling the data
under = RandomUnderSampler(sampling_strategy='majority') # 'majority' can be replaced by a float < 1
X_train_under,y_train_under = under.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after undersample is:{Counter(y_train_under)}')

 The count for y_train after undersample is:Counter({0: 1633, 1: 1633})


### Mehtod 3: Oversampling SMOTE
The synthetic minority oversampling technique (SMOTE) is another oversampling approach where new instances of minority class are interpolated. 

In [54]:
# Resampling using SMOTE the data
smote = SMOTE(sampling_strategy='auto') 
X_train_smote,y_train_smote = smote.fit_resample(X_train_scaled,y_train)
print(f' The count for y_train after SMOTE oversample is:{Counter(y_train_smote)}')

 The count for y_train after SMOTE oversample is:Counter({0: 6543, 1: 6543})


### Method 4: Cluster Centroid Umdersampling
The algorithm identifies clusters of the majority class, then generates synthetic data points, called centroids, that are representative of the clusters. The majority class is then undersampled down to the size of the minority class.

In [55]:
# # Resampling using Cluster Centroids
# cc = ClusterCentroids() 
# X_train_cc,y_train_cc = cc.fit_resample(X_train_scaled,y_train)
# print(f' The count for y_train after CC undersample is:{Counter(y_train_cc)}')

### Logistic Regression 

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [57]:
session_names = ['Imbalanced', 'Oversampling', 'Undersampling', 'SMOTE'] # CC is not running right now
X_train_sets = [X_train_scaled, X_train_over, X_train_under, X_train_smote]
y_train_sets = [y_train, y_train_over, y_train_under, y_train_smote]

In [58]:
# Run Logistic Regression Loop 

lr = LogisticRegression(solver='lbfgs',max_iter=100)

for session in range(len(X_train_sets)):

    lr.fit(X_train_sets[session], y_train_sets[session])
    y_pred = lr.predict(X_test_scaled)

    print(f'This is the result from {session_names[session]} method')
    print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
    # confusion_matrix = confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    print(classification_report(y_test, y_pred))
    print('------------------------------------')
    print('------------------------------------')



This is the result from Imbalanced method
accuracy: 0.886
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      2182
           1       0.77      0.61      0.68       544

    accuracy                           0.89      2726
   macro avg       0.84      0.78      0.80      2726
weighted avg       0.88      0.89      0.88      2726

------------------------------------
------------------------------------
This is the result from Oversampling method
accuracy: 0.885
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      2182
           1       0.67      0.84      0.74       544

    accuracy                           0.89      2726
   macro avg       0.81      0.87      0.84      2726
weighted avg       0.90      0.89      0.89      2726

------------------------------------
------------------------------------
This is the result from Undersampling method
accuracy: 0.884
              preci

### Naive Bayes Classification Modeling

In [59]:
# Run  NB Gaussian Loop

from sklearn.naive_bayes import GaussianNB

mnb = GaussianNB()

for session in range(len(X_train_sets)):

    mnb.fit(X_train_sets[session], y_train_sets[session])
    y_pred = mnb.predict(X_test_scaled)

    print(f'This is the result from {session_names[session]} method')
    print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
    # confusion_matrix = confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    print(classification_report(y_test, y_pred))
    print('------------------------------------')
    print('------------------------------------')


This is the result from Imbalanced method
accuracy: 0.901
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      2182
           1       0.76      0.73      0.75       544

    accuracy                           0.90      2726
   macro avg       0.85      0.84      0.84      2726
weighted avg       0.90      0.90      0.90      2726

------------------------------------
------------------------------------
This is the result from Oversampling method
accuracy: 0.895
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      2182
           1       0.68      0.91      0.77       544

    accuracy                           0.89      2726
   macro avg       0.83      0.90      0.85      2726
weighted avg       0.91      0.89      0.90      2726

------------------------------------
------------------------------------
This is the result from Undersampling method
accuracy: 0.888
              preci

### Support Vector Machine

In [60]:
# # Run  NB Gaussian Loop
# from sklearn.svm import SVC

# svm = SVC(kernel='linear') # other kernels : linear, rbf, sigmoid , Larger C more penalty for wrong classification

# for session in range(len(X_train_sets)):

#     svm.fit(X_train_sets[session], y_train_sets[session])
#     y_pred = svm.predict(X_test_scaled)

#     print(f'This is the result from {session_names[session]} method')
#     print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
#     # confusion_matrix = confusion_matrix(y_test, y_pred)
#     # print(confusion_matrix)
#     print(classification_report(y_test, y_pred))
#     print('------------------------------------')
#     print('------------------------------------')




### Decision Tree Algorithm

In [61]:
# Run Decision Tree Loop 

from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

for session in range(len(X_train_sets)):
    dtc.fit(X_train_sets[session],y_train_sets[session])
    y_pred = dtc.predict(X_test_scaled)

    print(f'This is the result from {session_names[session]} method')
    print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
    # confusion_matrix = confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    print(classification_report(y_test, y_pred))
    print('------------------------------------')
    print('------------------------------------')


This is the result from Imbalanced method
accuracy: 0.931
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2182
           1       0.82      0.83      0.83       544

    accuracy                           0.93      2726
   macro avg       0.89      0.89      0.89      2726
weighted avg       0.93      0.93      0.93      2726

------------------------------------
------------------------------------
This is the result from Oversampling method
accuracy: 0.933
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2182
           1       0.83      0.83      0.83       544

    accuracy                           0.93      2726
   macro avg       0.90      0.90      0.90      2726
weighted avg       0.93      0.93      0.93      2726

------------------------------------
------------------------------------
This is the result from Undersampling method
accuracy: 0.916
              preci

### Random Forest Classifier

In [62]:

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10, max_depth = 5)

for session in range(len(X_train_sets)):
    rfc.fit(X_train_sets[session],y_train_sets[session])
    y_pred = rfc.predict(X_test_scaled)

    print(f'This is the result from {session_names[session]} method')
    print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
    # confusion_matrix = confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    print(classification_report(y_test, y_pred))
    importances = rfc.feature_importances_
    print(f'The importances of the features are as follows:')
    print(list(sorted(zip(importances, X.columns), reverse=True)))
    print('------------------------------------')
    print('------------------------------------')

This is the result from Imbalanced method
accuracy: 0.953
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2182
           1       0.86      0.91      0.89       544

    accuracy                           0.95      2726
   macro avg       0.92      0.94      0.93      2726
weighted avg       0.95      0.95      0.95      2726

The importances of the features are as follows:
[(0.6983397562941086, 'Emp_Length'), (0.22740102594620265, 'Credit_Score'), (0.05615346154870566, 'Debt_income_ratio'), (0.01810575621098314, 'Loan_Amount')]
------------------------------------
------------------------------------
This is the result from Oversampling method
accuracy: 0.951
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2182
           1       0.85      0.92      0.88       544

    accuracy                           0.95      2726
   macro avg       0.91      0.94      0.93      2726
wei

### Decision Tree with ADABOOST

In [63]:

from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=10,learning_rate=0.5,max_depth=5)

for session in range(len(X_train_sets)):
    gbc.fit(X_train_sets[session],y_train_sets[session])
    y_pred = gbc.predict(X_test_scaled)
    print(f'This is the result from {session_names[session]} method')
    print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')
    # confusion_matrix = confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    print(classification_report(y_test, y_pred))
    print('------------------------------------')
    print('------------------------------------')

This is the result from Imbalanced method
accuracy: 0.957
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2182
           1       0.87      0.93      0.90       544

    accuracy                           0.96      2726
   macro avg       0.93      0.95      0.93      2726
weighted avg       0.96      0.96      0.96      2726

------------------------------------
------------------------------------
This is the result from Oversampling method
accuracy: 0.956
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      2182
           1       0.85      0.94      0.90       544

    accuracy                           0.96      2726
   macro avg       0.92      0.95      0.93      2726
weighted avg       0.96      0.96      0.96      2726

------------------------------------
------------------------------------
This is the result from Undersampling method
accuracy: 0.945
              preci

# Analysis 2. Model to predict interest rates

In [64]:
# Remove data where the columns which have more than 30% nulls values or more than 30% zeros

accepted_clean = raw_accepted.dropna(how='any', axis=1, thresh=file_accepted.shape[0]*0.3)

drop_cols = accepted_clean.columns[(accepted_clean == 0).sum() >0.3*accepted_clean.shape[0]]

accepted_clean = accepted_clean.drop(columns=drop_cols, axis=1)

# Drop additonal columns that do not have loan specific information

accepted_clean = accepted_clean.drop(columns=['id', 'url', 'zip_code','addr_state','last_pymnt_d','last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'policy_code',])

accepted_clean.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'dti',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'mths_since_last_delinq', 'open_acc', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'last_fico_range_high',
       'last_fico_range_low', 'application_type', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_last_12m', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_t

In [65]:
# Import Data dictionary to explain features
# dictionary = pd.read_csv('../Archive/dictionary.csv')
# dictionary = dictionary[['LoanStatNew','Description']]
# dictionary[dictionary['LoanStatNew'].apply(lambda x: x in accepted_clean.columns)]

,LoanStatNew,Description
1,acc_open_past_24mths,Number of trades opened in past 24 months.
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
6,application_type,Indicates whether the loan is an individual ap...
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/...
16,dti,A ratio calculated using the borrower�s total ...
18,earliest_cr_line,The month the borrower's earliest reported cre...
19,emp_length,Employment length in years. Possible values ar...


In [66]:
# Further removing columns that are not needed for interest prediction
target_col = accepted_clean['int_rate']

drop_cols = ['bc_open_to_buy','bc_util','earliest_cr_line', 'emp_title', 'funded_amnt', 'funded_amnt_inv', 'grade', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'last_fico_range_high', 'last_fico_range_low', 'loan_status', 'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mths_since_last_delinq', 'mths_since_rcnt_il','mths_since_recent_bc', 'mths_since_recent_revol_delinq', 'mths_since_recent_revol_delinq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'open_il_24m', 'open_act_il', 'open_rv_12m', 'open_rv_24m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'purpose', 'pymnt_plan', 'sub_grade', 'term', 'title', 'total_bal_ex_mort', 'total_bal_il', 'total_bc_limit', 'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_prncp', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']

accepted_clean = accepted_clean.drop(columns=drop_cols, axis=1)
#accepted_clean.to_csv('../Archive/clean_accepted_with_columns.csv', index=False)

In [67]:
accepted_clean.head()

,loan_amnt,emp_length,home_ownership,annual_inc,verification_status,dti,fico_range_low,fico_range_high,open_acc,revol_bal,...,all_util,total_rev_hi_lim,inq_last_12m,acc_open_past_24mths,avg_cur_bal,mo_sin_rcnt_tl,mths_since_recent_inq,num_sats,num_tl_op_past_12m,tot_hi_cred_lim
0,3600.00000,10+ years,MORTGAGE,55000.00000,Not Verified,5.91000,675.00000,679.00000,7.00000,2765.00000,...,34.00000,9300.00000,4.00000,4.00000,20701.00000,3.00000,4.00000,7.00000,3.00000,178050.00000
1,24700.00000,10+ years,MORTGAGE,65000.00000,Not Verified,16.06000,715.00000,719.00000,22.00000,21470.00000,...,29.00000,111800.00000,6.00000,4.00000,9733.00000,2.00000,0.00000,22.00000,2.00000,314017.00000
2,20000.00000,10+ years,MORTGAGE,63000.00000,Not Verified,10.78000,695.00000,699.00000,6.00000,7869.00000,...,65.00000,14000.00000,1.00000,6.00000,31617.00000,14.00000,10.00000,6.00000,0.00000,218418.00000
3,35000.00000,10+ years,MORTGAGE,110000.00000,Source Verified,17.06000,785.00000,789.00000,13.00000,7802.00000,...,45.00000,67300.00000,0.00000,2.00000,23192.00000,2.00000,NaN,13.00000,1.00000,381215.00000
4,10400.00000,3 years,MORTGAGE,104433.00000,Source Verified,25.37000,695.00000,699.00000,12.00000,21929.00000,...,78.00000,34000.00000,3.00000,10.00000,27644.00000,4.00000,1.00000,12.00000,4.00000,439570.00000


In [68]:
accepted_clean.dtypes

loan_amnt                float64
emp_length                object
home_ownership            object
annual_inc               float64
verification_status       object
dti                      float64
fico_range_low           float64
fico_range_high          float64
open_acc                 float64
revol_bal                float64
revol_util               float64
total_acc                float64
application_type          object
tot_cur_bal              float64
open_acc_6m              float64
il_util                  float64
all_util                 float64
total_rev_hi_lim         float64
inq_last_12m             float64
acc_open_past_24mths     float64
avg_cur_bal              float64
mo_sin_rcnt_tl           float64
mths_since_recent_inq    float64
num_sats                 float64
num_tl_op_past_12m       float64
tot_hi_cred_lim          float64
dtype: object

In [75]:
accepted_clean['emp_length'] = accepted_clean['emp_length'].replace('10+ years','10')
accepted_clean['emp_length'] = accepted_clean['emp_length'].replace('< 1 year','0')
accepted_clean['emp_length'] = accepted_clean['emp_length'].replace('1 year','1')
accepted_clean['emp_length'] = accepted_clean['emp_length'].str.replace(' years','')
accepted_clean['emp_length'] = accepted_clean['emp_length'].replace(' ','')
accepted_clean['emp_length'] = pd.to_numeric(accepted_clean['emp_length'])
accepted_clean['emp_length'].value_counts()

10.00000    748005
2.00000     203677
0.00000     189988
3.00000     180753
1.00000     148403
5.00000     139698
4.00000     136605
6.00000     102628
7.00000      92695
8.00000      91914
9.00000      79395
Name: emp_length, dtype: int64

In [76]:
accepted_clean = pd.get_dummies(accepted_clean)

Index(['loan_amnt', 'emp_length', 'annual_inc', 'dti', 'fico_range_low',
       'fico_range_high', 'open_acc', 'revol_bal', 'revol_util', 'total_acc',
       'tot_cur_bal', 'open_acc_6m', 'il_util', 'all_util', 'total_rev_hi_lim',
       'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'mo_sin_rcnt_tl',
       'mths_since_recent_inq', 'num_sats', 'num_tl_op_past_12m',
       'tot_hi_cred_lim', 'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN',
       'home_ownership_RENT', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'application_type_Individual', 'application_type_Joint App'],
      dtype='object')

In [88]:
accepted_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 34 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   loan_amnt                            float64
 1   emp_length                           float64
 2   annual_inc                           float64
 3   dti                                  float64
 4   fico_range_low                       float64
 5   fico_range_high                      float64
 6   open_acc                             float64
 7   revol_bal                            float64
 8   revol_util                           float64
 9   total_acc                            float64
 10  tot_cur_bal                          float64
 11  open_acc_6m                          float64
 12  il_util                              float64
 13  all_util                             float64
 14  total_rev_hi_lim                     float64
 15  inq_last_12m                    

In [80]:
# Split into test and train and preprocess the data
X = accepted_clean.dropna()
y = target_col.dropna()
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [92]:
import tensorflow as tf

layer_1 = 20
inputs = X_train.shape[1]
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=layer_1, activation='relu', input_dim=inputs))
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                700       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________


In [93]:
fit_model =nn_model.fit(X_train, y_train, epochs=50)

Train on 1695525 samples
Epoch 1/50
1695525/1695525 [==============================] - 372s 219us/sample - loss: nan - accuracy: 0.0000e+00
Epoch 2/50
  78080/1695525 [>.............................] - ETA: 5:13 - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 